In [16]:
import numpy as np
import wandb
import yaml
import shutil
from yaml.loader import SafeLoader
from datetime import datetime
import torch
import torch.nn as nn
import torch.optim as optim
import torch.distributions as td
import torchvision
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms, datasets
from sklearn.model_selection import train_test_split
from netAE import AE
from netVAE import VAE

In [13]:
path_yaml  = "config.yaml"

#config = yaml.load(path_yaml,Loader=SafeLoader)

In [14]:
data = np.load("data4D.npy", allow_pickle= True)
data.shape

(5000, 3, 32, 32)

In [15]:
# Trainset/testset
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# 将NumPy数组转换为张量
train_data_tensor = torch.from_numpy(train_data)
test_data_tensor = torch.from_numpy(test_data)

# 创建DataLoader
batch_size = 128
train_dataset = TensorDataset(train_data_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = TensorDataset(test_data_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [17]:
class VAE(nn.Module):
    def __init__(self, img_channels, latent_dim):
        super(VAE, self).__init__()

        # 编码器
        self.encoder = nn.Sequential(
            nn.Conv2d(img_channels, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.Flatten()
        )

        # 获取编码器的输出大小
        self.encoder_output_size = self.encoder(torch.zeros(1, img_channels, 32, 32)).shape[1]

        # 计算均值和对数方差
        self.fc_mu = nn.Linear(self.encoder_output_size, latent_dim)
        self.fc_logvar = nn.Linear(self.encoder_output_size, latent_dim)

        # 解码器
        self.fc_decode = nn.Linear(latent_dim, self.encoder_output_size)
        self.decoder = nn.Sequential(
            nn.Unflatten(1, (128, 4, 4)),
            nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, kernel_size=4, stride=2, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(32, img_channels, kernel_size=4, stride=2, padding=1),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        h = self.encoder(x)
        mu, logvar = self.fc_mu(h), self.fc_logvar(h)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decoder(self.fc_decode(z))
        return x_recon, mu, logvar

In [18]:
def vae_loss(x, x_recon, mu, logvar):
    recon_loss = nn.functional.mse_loss(x_recon, x, reduction='sum') / x.size(0)
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / x.size(0)
    return recon_loss + kl_div

In [20]:
img_channels = 3
latent_dim = 128
model = VAE(img_channels, latent_dim)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [21]:
num_epochs = 50

# TRAIN
model.train()

for epoch in range(num_epochs):
    train_loss = 0.0
    for batch_idx, (data,) in enumerate(train_loader):
        data = data.float()
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = vae_loss(data, recon_batch, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()

    train_loss /= len(train_loader.dataset)
    print('====> Epoch: {} Average loss: {:.4f}'.format(epoch + 1, train_loss))

====> Epoch: 1 Average loss: 3.2395
====> Epoch: 2 Average loss: 1.3518
====> Epoch: 3 Average loss: 1.3372
====> Epoch: 4 Average loss: 1.3373
====> Epoch: 5 Average loss: 1.3263
====> Epoch: 6 Average loss: 1.3258
====> Epoch: 7 Average loss: 1.3200
====> Epoch: 8 Average loss: 1.3240
====> Epoch: 9 Average loss: 1.3227
====> Epoch: 10 Average loss: 1.3154
====> Epoch: 11 Average loss: 1.3174
====> Epoch: 12 Average loss: 1.3194
====> Epoch: 13 Average loss: 1.3154
====> Epoch: 14 Average loss: 1.3137
====> Epoch: 15 Average loss: 1.3133
====> Epoch: 16 Average loss: 1.3093
====> Epoch: 17 Average loss: 1.3048
====> Epoch: 18 Average loss: 1.3003
====> Epoch: 19 Average loss: 1.2967
====> Epoch: 20 Average loss: 1.2919
====> Epoch: 21 Average loss: 1.2875
====> Epoch: 22 Average loss: 1.2889
====> Epoch: 23 Average loss: 1.2917
====> Epoch: 24 Average loss: 1.2945
====> Epoch: 25 Average loss: 1.2896
====> Epoch: 26 Average loss: 1.2876
====> Epoch: 27 Average loss: 1.2896
====> Epoc

In [22]:
model.eval()
test_loss = 0.0

with torch.no_grad():
    for i, (data,) in enumerate(test_loader):
        data = data.float()
        recon_batch, mu, logvar = model(data)
        test_loss += vae_loss(data, recon_batch, mu, logvar).item()

test_loss /= len(test_loader.dataset)
print('====> Test set loss: {:.4f}'.format(test_loss))

====> Test set loss: 1.2608
